In [10]:
import pandas as pd
import nltk
import tgt
import os
import random
import spacy
import torch
print(dir(tgt.Tier))
root_dir = './'
word_level_timing = root_dir+'word_level_timing'
motion_label = root_dir+'motion_labels'

['__class__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_annotation_index_by_end_time', '_get_annotation_index_by_start_time', '_get_annotation_index_range_between_timepoints', '_get_annotation_indices_by_time', '_get_annotations', '_get_end_time', '_get_start_time', '_set_end_time', '_set_start_time', 'add_annotation', 'add_annotations', 'annotations', 'delete_annotation_by_end_time', 'delete_annotation_by_start_time', 'delete_annotations_between_timepoints', 'delete_annotations_by_time', 'delete_annotations_with_text', 'delete_empty_annotations', 'end_time', 'get_annotation_by_end_time', 'get_annotation_by_start_time', 'get_ann

In [6]:
german_text = "Die Sonne scheint am Himmel und die Vögel singen."
german_tagger = spacy.load("de_core_news_sm")
doc = german_tagger(german_text)
print(doc)
# Extract tokens and POS tags
# for token in doc:
#     print(token.text, token.pos_)

Die Sonne scheint am Himmel und die Vögel singen.


In [7]:
def get_all_textgrid_files(path):
    filenames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".TextGrid"):
                filenames.append(os.path.join(root, file))
    return filenames
word_level_timing_annotation = get_all_textgrid_files(word_level_timing)
print(word_level_timing_annotation)      

['./word_level_timing/r1A_wordlevel.TextGrid', './word_level_timing/r13A_wordlevel.TextGrid', './word_level_timing/r3A_wordlevel.TextGrid', './word_level_timing/r19B_wordlevel.TextGrid', './word_level_timing/r15B_wordlevel.TextGrid', './word_level_timing/r11B_wordlevel.TextGrid', './word_level_timing/r19A_wordlevel.TextGrid', './word_level_timing/r2B_wordlevel.TextGrid', './word_level_timing/r3B_wordlevel.TextGrid', './word_level_timing/r12B_wordlevel.TextGrid', './word_level_timing/r9A_wordlevel.TextGrid', './word_level_timing/r10A_wordlevel.TextGrid', './word_level_timing/r9B_wordlevel.TextGrid', './word_level_timing/r18A_wordlevel.TextGrid', './word_level_timing/r17B_wordlevel.TextGrid', './word_level_timing/r8B_wordlevel.TextGrid', './word_level_timing/r15A_wordlevel.TextGrid', './word_level_timing/r5A_wordlevel.TextGrid', './word_level_timing/r12A_wordlevel.TextGrid', './word_level_timing/r7B_wodlevel.TextGrid', './word_level_timing/r6B_wordleve.TextGrid', './word_level_timing/r5B

In [20]:
pos_tag = []
for index, f in enumerate(word_level_timing_annotation):
    if f == './word_level_timing/r10B_wordlevel.TextGrid':
        print(f'processing file {f}')
        textgrid= tgt.read_textgrid(f)
        tier_names = textgrid.get_tier_names()
        for names in tier_names:
            text_tier = textgrid.get_tier_by_name(names)
            for annotation in text_tier.annotations:
                doc = german_tagger(annotation.text)
                for token in doc:
                    pos_tag.append((token.text, token.pos_, str(f.split('/')[2]).split('.')[0].split('_')[0], names, annotation.start_time, annotation.end_time))
        dataframe = pd.DataFrame(pos_tag, columns=['text', 'pos_tag', 'session', 'tier_name', 'start_time', 'end_time'])
        dataframe.to_csv('./pos_tagging/'+str(f.split('/')[2].split('.')[0])+'.csv')

processing file ./word_level_timing/r10B_wordlevel.TextGrid


ValueError: Could not add object Interval(469.468776, 469.920272, "n") to this tier: Overlap.

### Need to define a dataset class first 

In [ ]:
class WordLevelTiming(torch.utils.data.Dataset):
    pass

In [ ]:
import torch
class RNNModel(torch.nn.Module):
    def __init__(self, hidden_dim, emdbedding_diminput_shape):
        super().__init__()
        self.hidden_dim = hidden_dim
        # self.word_embedding = 
        # self.lstm1 = nn.LSTM(embedding_dim, hidden_dim)
    def forward(self, x)
        # embeds = self.word_embedding
        # lstm_out, _ = self.lstm(embeds.view(len(x), 1, -1))
        # tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        # tag_scores = F.log_softmax(tag_space, dim=1)
        # return tag_scores

In [ ]:
import time
import torch
def train(train_loader,val_loader, model, loss_function, optimizer, epochs, device):
    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    history = {}
    history['loss'] = []
    history['val_loss'] = []
    history['acc'] = []
    history['val_acc'] = []
    start_time_sec = time.time()
    for epoch in range(epochs):
        model=model.to(device)
        model.train()
        train_loss = 0.0
        num_train_correct = 0
        num_train_examples = 0
        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()
            
            images = batch['image'].to(device)
            target = batch['label'].to(device)

            output = model(images.float())
            loss = loss_function(output, target.float())
            
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()*images.size(0)
            num_train_correct  += (torch.max(output, 1)[1] == target).sum().item()
            num_train_examples += images.shape[0]
        train_acc =  num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_loader.dataset)

        # --- EVALUATE ON VALIDATION SET -------------------------------------

        model.eval()
        val_loss       = 0.0
        num_val_correct  = 0
        num_val_examples = 0

        for batch in val_loader:
            images = batch['image'].to(device)
            target = batch['label'].to(device)

            output = model(images.float())
            loss = loss_function(output, target.float())

            val_loss         += loss.data.item()*images.size(0)
            num_val_correct  += (torch.max(output, 1)[1] == target).sum().item()
            num_val_examples += images.shape[0]
        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_loader.dataset)
        print('Epoch %3d/%3d, train loss: %5.2f, train acc: %5.2f, val loss: %5.2f, val acc: %5.2f' % \
              (epoch+1, epochs, train_loss, train_acc, val_loss, val_acc))

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['acc'].append(train_acc)
        history['val_acc'].append(val_acc)

         # END OF TRAINING LOOP
    end_time_sec       = time.time()
    total_time_sec     = end_time_sec - start_time_sec
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))

    return history